In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
location="D:\s_chernov\AnacondaProjects\MLSSZ\Data\\"
df_train = pd.read_csv(location+"train.csv",sep=';',na_values="None")
df_test = pd.read_csv(location+"test.csv",sep=';',na_values="None")

In [3]:
del df_train['id']


In [4]:
df_train['age_year']=np.int16(df_train.age/365)
df_test['age_year']=np.int16(df_test.age/365)
df_train.loc[df_train.weight<20,'weight']=df_train.loc[df_train.weight<20,'weight']*10
df_test.loc[df_test.weight<20,'weight']=df_test.loc[df_test.weight<20,'weight']*10
df_train.loc[df_train.height<100,'height']=df_train.loc[df_train.height<100,'height']*10
df_test.loc[df_test.height<100,'height']=df_test.loc[df_test.height<100,'height']*10
#height and weight errors
correct=df_train.weight>df_train.height
correct_weight=df_train.loc[correct,'weight']
correct_height=df_train.loc[correct,'height']
df_train.loc[correct,'weight']=correct_height
df_train.loc[correct,'height']=correct_weight

correct=df_test.weight>df_test.height
correct_weight=df_test.loc[correct,'weight']
correct_height=df_test.loc[correct,'height']
df_test.loc[correct,'weight']=correct_height
df_test.loc[correct,'height']=correct_weight

#____ap_lo and ap_hi
df_train.loc[df_train.ap_lo<0,'ap_lo']=-df_train.loc[df_train.ap_lo<0,'ap_lo']
df_test.loc[df_test.ap_lo<0,'ap_lo']=-df_test.loc[df_test.ap_lo<0,'ap_lo']

df_train.loc[df_train.ap_lo>6000,'ap_lo']=df_train.loc[df_train.ap_lo>6000,'ap_lo']/100
df_train.loc[df_train.ap_lo>600,'ap_lo']=df_train.loc[df_train.ap_lo>600,'ap_lo']/10
df_test.loc[df_test.ap_lo>6000,'ap_lo']=df_test.loc[df_test.ap_lo>6000,'ap_lo']/100
df_test.loc[df_test.ap_lo>600,'ap_lo']=df_test.loc[df_test.ap_lo>600,'ap_lo']/10
df_train.loc[df_train.ap_lo<=10,'ap_lo']=df_train.loc[df_train.ap_lo<=10,'ap_lo']*10
df_test.loc[df_test.ap_lo<=10,'ap_lo']=df_test.loc[df_test.ap_lo<=10,'ap_lo']*10

df_train.loc[df_train.ap_hi<0,'ap_hi']=-df_train.loc[df_train.ap_hi<0,'ap_hi']
df_test.loc[df_test.ap_hi<0,'ap_hi']=-df_test.loc[df_test.ap_hi<0,'ap_hi']

df_train.loc[df_train.ap_hi>=10000,'ap_hi']=df_train.loc[df_train.ap_hi>=10000,'ap_hi']/100
df_test.loc[df_test.ap_hi>=10000,'ap_hi']=df_test.loc[df_test.ap_hi>=10000,'ap_hi']/100
df_train.loc[df_train.ap_hi>=300,'ap_hi']=df_train.loc[df_train.ap_hi>=300,'ap_hi']/10
df_test.loc[df_test.ap_hi>=300,'ap_hi']=df_test.loc[df_test.ap_hi>=300,'ap_hi']/10

df_train.loc[df_train.ap_hi<30,'ap_hi']=df_train.loc[df_train.ap_hi<30,'ap_hi']*10
df_test.loc[df_test.ap_hi<30,'ap_hi']=df_test.loc[df_test.ap_hi<30,'ap_hi']*10

correct=df_train.ap_hi<df_train.ap_lo
correct_hi=df_train.loc[correct,'ap_hi']
correct_lo=df_train.loc[correct,'ap_lo']
df_train.loc[correct,'ap_hi']=correct_lo
df_train.loc[correct,'ap_lo']=correct_hi

correct=df_test.ap_hi<df_test.ap_lo
correct_hi=df_test.loc[correct,'ap_hi']
correct_lo=df_test.loc[correct,'ap_lo']
df_test.loc[correct,'ap_hi']=correct_lo
df_test.loc[correct,'ap_lo']=correct_hi

In [5]:
#df_train['age_year']=np.int16(df_train.age/365)
#df_test['age_year']=np.int16(df_test.age/365)
#df_train['ap_div']=df_train.ap_lo/df_train.ap_hi
#df_test['ap_div']=df_test.ap_lo/df_train.ap_hi
#df_train['fat']=df_train.weight/(((df_train.height)/100)**2)
#df_test['fat']=df_test.weight/(((df_train.height)/100)**2)
#df_train['diff_weight']=50+0.75*(df_train.height-150) + (df_train.age_year-20)/4-df_train.weight
#df_test['diff_weight']=50+0.75*(df_test.height-150) + (df_test.age_year-20)/4-df_test.weight
#df_train['age_weight']=df_train.age/df_train.weight
#df_test['age_weight']=df_test.age/df_test.weight

In [6]:
df_test.alco=df_test.alco.fillna(0)
df_test.smoke=df_test.smoke.fillna(0)
df_test.active=df_test.active.fillna(1)
features=list(df_train.columns)
features.remove('cardio')


In [7]:
xgb_params = {'colsample_bytree': 0.6,
 'eval_metric': 'logloss',
 'gamma': 0.4,
 'learning_rate': 0.02,
 'max_depth': 3,
 'min_child_weight': 5,
 'n_estimators': 1000,
 'objective': 'binary:logistic',
 'reg_alpha': 0.01,
 'seed': 123,
 'silent': 1,
 'subsample': 0.9}

#cvresult = xgb.cv(xgb_params, dtrain, num_boost_round=1000, nfold=5,early_stopping_rounds=50, seed=123)

In [8]:
def xgb_factory(xgb_params):
    return( xgb.XGBClassifier(
             learning_rate=xgb_params['learning_rate'],
             n_estimators=xgb_params['n_estimators'],
             max_depth=xgb_params['max_depth'],
             min_child_weight=xgb_params['min_child_weight'],
             gamma=xgb_params['gamma'],
             subsample=xgb_params['subsample'],
             colsample_bytree=xgb_params['colsample_bytree'],
             objective=xgb_params['objective'],
             nthread=4,
             scale_pos_weight=1,
             seed=xgb_params['seed']))
       

In [9]:
from sklearn.cross_validation import StratifiedKFold
cv = StratifiedKFold(df_train.cardio, n_folds=3, shuffle=True, random_state=123)

In [18]:
def makeCV(xgb_params,param_grid):
    hyper = xgb_factory(xgb_params)
    alg_xgb_grid = GridSearchCV(hyper, param_grid,cv=cv, scoring='neg_log_loss',refit=True, verbose=1, n_jobs=4)
    alg_xgb_grid.fit(df_train.loc[:,features],df_train.cardio)
    alg_xgb_best = alg_xgb_grid.best_estimator_
    print("Accuracy (xgboost auto): {} with params {}"
      .format(alg_xgb_grid.best_score_, alg_xgb_grid.best_params_))

    return alg_xgb_grid.best_params_


In [19]:
def tuneDepth(xgb_params):
    param_grid = {
             'max_depth':range(3,5,1),
             'min_child_weight':range(5,9,1)
    }
    best_params=makeCV(xgb_params,param_grid)
    xgb_params['max_depth']=best_params['max_depth']
    xgb_params['min_child_weight']=best_params['min_child_weight']
    return xgb_params

def tuneGamma(xgb_params):
    param_grid = {
         'gamma':[i/10.0 for i in range(0,5)]
        }
    best_params=makeCV(xgb_params,param_grid)

    xgb_params['gamma']=best_params['gamma']
    return xgb_params


def tuneSample(xgb_params):
   
    param_grid = {
         'subsample':[i/10.0 for i in range(6,10)],
         'colsample_bytree':[i/10.0 for i in range(6,10)]
        }

    best_params=makeCV(xgb_params,param_grid)


    xgb_params['subsample']=best_params['subsample']
    xgb_params['colsample_bytree']=best_params['colsample_bytree']
    return xgb_params

def tuneAlpha(xgb_params):
   
    param_grid = {
         'reg_alpha':[0.01, 0.015, 0.02, 0.025]
        }
    best_params=makeCV(xgb_params,param_grid)
    xgb_params['reg_alpha']=best_params['reg_alpha']
    return xgb_params

def tuneLearn(xgb_params):
    param_grid =  {"n_estimators": [1000, 1100, 900],
        "learning_rate": [0.01, 0.02, 0.03]}

    best_params=makeCV(xgb_params,param_grid)

    xgb_params['n_estimators']=best_params['n_estimators']
    xgb_params['learning_rate']=best_params['learning_rate']
    return xgb_params

In [10]:
from sklearn.model_selection import GridSearchCV

#tune max_depth, min_child_weight
hyper = xgb_factory(xgb_params)
param_grid = {
    
 'max_depth':range(3,8,1),
 'min_child_weight':range(1,6,2)
}

alg_xgb_grid = GridSearchCV(hyper, param_grid,cv=cv, scoring='neg_log_loss',refit=True, verbose=1, n_jobs=4)
alg_xgb_grid.fit(df_train.loc[:,features],df_train.cardio)
alg_xgb_best = alg_xgb_grid.best_estimator_
print("Accuracy (xgboost auto): {} with params {}"
      .format(alg_xgb_grid.best_score_, alg_xgb_grid.best_params_))

xgb_params['max_depth']=alg_xgb_grid.best_params_['max_depth']
xgb_params['min_child_weight']=alg_xgb_grid.best_params_['min_child_weight']

#Accuracy (xgboost auto): -0.540365372589 with params {'max_depth': 3, 'min_child_weight': 5}
#Accuracy (xgboost auto): -0.538897852082 with params {'max_depth': 3, 'min_child_weight': 5}

Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed: 13.1min finished


Accuracy (xgboost auto): -0.538897852082 with params {'max_depth': 3, 'min_child_weight': 5}


In [11]:
#tune gamma
hyper = xgb_factory(xgb_params)

param_grid2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
alg_xgb_grid = GridSearchCV(hyper, param_grid2,cv=cv, scoring='neg_log_loss',refit=True, verbose=1, n_jobs=4)
alg_xgb_grid.fit(df_train.loc[:,features],df_train.cardio)
alg_xgb_best = alg_xgb_grid.best_estimator_
print("Accuracy (xgboost autтакo): {} with params {}"
      .format(alg_xgb_grid.best_score_, alg_xgb_grid.best_params_))

xgb_params['gamma']=alg_xgb_grid.best_params_['gamma']

#Accuracy (xgboost autтакo): -0.539870740274 with params {'gamma': 0.4}
#Accuracy (xgboost autтакo): -0.538893119855 with params {'gamma': 0.2}

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:  3.0min finished


Accuracy (xgboost autтакo): -0.538893119855 with params {'gamma': 0.2}


In [12]:
hyper = xgb_factory(xgb_params)

param_grid3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

alg_xgb_grid = GridSearchCV(hyper, param_grid3,cv=cv, scoring='neg_log_loss',refit=True, verbose=1, n_jobs=4)
alg_xgb_grid.fit(df_train.loc[:,features],df_train.cardio)
alg_xgb_best = alg_xgb_grid.best_estimator_
print("Accuracy (xgboost auto): {} with params {}"
      .format(alg_xgb_grid.best_score_, alg_xgb_grid.best_params_))

xgb_params['subsample']=alg_xgb_grid.best_params_['subsample']
xgb_params['colsample_bytree']=alg_xgb_grid.best_params_['colsample_bytree']

#Accuracy (xgboost auto): -0.540194133887 with params {'subsample': 0.9, 'colsample_bytree': 0.6}
#Accuracy (xgboost auto): -0.538893119855 with params {'subsample': 0.9, 'colsample_bytree': 0.6}

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=4)]: Done  48 out of  48 | elapsed: 10.6min finished


Accuracy (xgboost auto): -0.538893119855 with params {'subsample': 0.9, 'colsample_bytree': 0.6}


In [13]:
hyper = xgb_factory(xgb_params)
param_grid4 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}

alg_xgb_grid = GridSearchCV(hyper, param_grid4,cv=cv, scoring='neg_log_loss',refit=True, verbose=1, n_jobs=4)
alg_xgb_grid.fit(df_train.loc[:,features],df_train.cardio)
alg_xgb_best = alg_xgb_grid.best_estimator_
print("Accuracy (xgboost auto): {} with params {}"
      .format(alg_xgb_grid.best_score_, alg_xgb_grid.best_params_))

xgb_params['reg_alpha']=alg_xgb_grid.best_params_['reg_alpha']

#Accuracy (xgboost auto): -0.5401745397 with params {'reg_alpha': 0.01}
#ccuracy (xgboost auto): -0.538888944809 with params {'reg_alpha': 0.01}

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:  3.1min finished


Accuracy (xgboost auto): -0.538888944809 with params {'reg_alpha': 0.01}


In [16]:
param_grid4 =  {"n_estimators": [1000, 2000, 3000],
    "learning_rate": [0.01, 0.02, 0.05]}

alg_xgb_grid = GridSearchCV(hyper, param_grid4,cv=cv, scoring='neg_log_loss',refit=True, verbose=1, n_jobs=4)
alg_xgb_grid.fit(df_train.loc[:,features],df_train.cardio)
alg_xgb_best = alg_xgb_grid.best_estimator_
print("Accuracy (xgboost auto): {} with params {}"
      .format(alg_xgb_grid.best_score_, alg_xgb_grid.best_params_))

xgb_params['n_estimators']=alg_xgb_grid.best_params_['n_estimators']
xgb_params['learning_rate']=alg_xgb_grid.best_params_['learning_rate']

#Accuracy (xgboost auto): -0.538893119855 with params {'n_estimators': 1000, 'learning_rate': 0.02}

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed: 10.6min finished


Accuracy (xgboost auto): -0.538893119855 with params {'n_estimators': 1000, 'learning_rate': 0.02}


In [ ]:
hyper = xgb_factory(xgb_params)


In [ ]:
kfold = KFold(n_splits=10, random_state=7)
#results = cross_val_score(hyper, df_train.loc[:,features], df_train.cardio, cv=kfold,scoring='neg_log_loss',n_jobs=-1)
results = cross_val_score(hyper, df_train.loc[:,features], df_train.cardio, cv=kfold,scoring='neg_log_loss',n_jobs=-1)
print results.mean(),results.std()

fit=hyper.fit(df_train.loc[:,features],df_train.cardio,eval_metric='logloss')

#model = xgb.train(xgb_params,dtrain_all, num_boost_round=num_boost_round)
#pred = model.predict(dtest)
pred = fit.predict_proba(df_test.loc[:,features])[:,1]

In [ ]:
feat_imp = pd.Series(hyper.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')
plt.show()

In [ ]:
res=pd.DataFrame({'cardio':pred},columns=['id','cardio'])
res.cardio.to_csv(location+'submission.csv',index=False,header=False)

In [17]:
xgb_params

{'colsample_bytree': 0.6,
 'eval_metric': 'logloss',
 'gamma': 0.2,
 'learning_rate': 0.02,
 'max_depth': 3,
 'min_child_weight': 5,
 'n_estimators': 1000,
 'objective': 'binary:logistic',
 'reg_alpha': 0.01,
 'seed': 123,
 'silent': 1,
 'subsample': 0.9}

In [ ]:
xgb_params={'colsample_bytree': 0.6,
 'eval_metric': 'logloss',
 'gamma': 0.2,
 'learning_rate': 0.02,
 'max_depth': 3,
 'min_child_weight': 5,
 'n_estimators': 1000,
 'objective': 'binary:logistic',
 'reg_alpha': 0.01,
 'seed': 123,
 'silent': 1,
 'subsample': 0.9}

In [20]:
xgb_params=tuneDepth(xgb_params)
xgb_params=tuneGamma(xgb_params)
xgb_params=tuneSample(xgb_params)
xgb_params=tuneAlpha(xgb_params)
xgb_params=tuneLearn(xgb_params)


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed:  6.6min finished


Accuracy (xgboost auto): -0.538830822235 with params {'max_depth': 3, 'min_child_weight': 7}
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:  3.0min finished


Accuracy (xgboost auto): -0.538824325743 with params {'gamma': 0.1}
Fitting 3 folds for each of 16 candidates, totalling 48 fits


...

Accuracy (xgboost auto): -0.538824325743 with params {'subsample': 0.9, 'colsample_bytree': 0.6}
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:  2.4min finished


Accuracy (xgboost auto): -0.538819940798 with params {'reg_alpha': 0.015}
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed:  5.2min finished


Accuracy (xgboost auto): -0.538820655103 with params {'n_estimators': 900, 'learning_rate': 0.02}


In [22]:
xgb_params

{'colsample_bytree': 0.6,
 'eval_metric': 'logloss',
 'gamma': 0.1,
 'learning_rate': 0.02,
 'max_depth': 3,
 'min_child_weight': 7,
 'n_estimators': 900,
 'objective': 'binary:logistic',
 'reg_alpha': 0.015,
 'seed': 123,
 'silent': 1,
 'subsample': 0.9}